# 1. A Brief History of PyTorch

<img src="./images/torch.png" alt="Torch logo" title="Torch" width="20%" height="20%" style="margin:0px 5px" /></img>
2002年10月，Torch发布了第一个release，这是一个开源的机器学习库，提供了一种科学计算框架，以及一种基于Lua的脚本语言。Torch中包含了nn模块，专门用于构建神经网络用，包含了大量类似forward()、backward()、nn.Sequential()等深度学习常用函数接口。[注1](https://en.wikipedia.org/wiki/Torch_(machine_learning))
<br />
<br />
<br />
<img src="./images/pytorch.png#" alt="PyTorch logo" title="PyTorch" width="25%" height="25%" style="margin:0px 5px"/></img>
2016年10月，Facebook启动了内部项目：PyTorch，它使用了Torch的库，其开发者是Adam Paszke，而Adam Paszke的师傅是Soumith Chintala，Torch的核心开发者之一。因此，PyTorch项目一开始就与Torch项目有着紧密的联系。[注2](https://en.wikipedia.org/wiki/PyTorch)
<br />
<br />
<img src="./images/pytorch_x_caffe2.png#" alt="pytorch_x_caffe2 logo" title="pytorch_x_caffe2" width="50%" height="50%" style="margin:0px 0px" /></img>
2017年4月，Facebook发布了caffe2，依然由caffe的创始人贾扬清操刀，专注于大规模分布式训练、移动端推理、新硬件的支持、模型量化等方向。自此，Facebook在深度学习框架领域同时拥有了pytorch和caffe2两大利器，前者专注于科研学术领域，后者专注与生产部署领域。但是好景不长，2018年3月，caffe2被合并入PyTorch项目，caffe2作为独立项目停止维护。2018年5月，PyTorch 1.0上线，自此，PyTorch成为兼顾科研和生产的统一平台。[注3](https://caffe2.ai/blog/2018/05/02/Caffe2_PyTorch_1_0.html),[注4](https://medium.com/@Synced/caffe2-merges-with-pytorch-a89c70ad9eb7)
<br />
<br />

<img src="./images/onnx.png#" alt="ONNX logo" title="ONNX" width="25%" height="25%" style="margin:0px 0px" /></img>
同样是在2017年，Facebook和微软联合发布了一款开放式模型文件格式：ONNX(Open Neural Network Exchange)，用于解决不同深度学习框架下，不同格式的模型无法相互兼容匹配的问题。PyTorch支持将自己模型文件转为ONNX格式的，进而可以平滑运行在其它的深度框架上，比如MXNET，CNTK，TensorRT，甚至TensorFlow。


<br />

**<font font-weight="bold" color=blue size=4>上面啰嗦这么多，其实都或多或少的体现在libtorch的源码上，先搂一眼1.7.1版本源码的目录结构：</font>**

# 2. PyTorch 1.7.1源码目录结构

![pytorch source code files](./images/pytorch_code_structure.png)

"持续补充中，欢迎纠错"
<br />
<br />

***重点需要关注的部分：***

## aten

`aten(A TENsor library):  Tensor的底层定义和操作函数（含神经网络相关运算），含CPU和GPU端 ` 

- aten/ATen - aten下面最重要的组成部分，原名TensorLib     
    - **aten/ATen/native - ATen的底层核心，用于定义算子和相关函数，这些函数可以被Python或者c++的上层API调用；本部分代码还含有对多种硬件平台的支持，以及量化相关操作的底层实现**       
    - aten/ATen/core - ATen的最小子集，用于移动端部署；      
    - aten/ATen/quantized - 模型量化的上层接口；     
- aten/TH - CPU张量运算库    
- aten/THC - CUDA张量运算库    
- aten/THCUNN - CUDA神经网络运算库    

已经弃用的代码模块：  
- ~~*aten/THCS - torch cuda sparse*~~     
- ~~*aten/THNN - torch neural network*~~      
- ~~*aten/THS - torch sparse*~~      

按知乎网友的描述，这些TH打头的功能模块，都是继承自老Torch库的命名方式，随着Torch库的逐渐淡化，这些功能模块都在逐渐搬移到ATen中去，可以预见，也许在不远的将来，ATen会实现底层代码的大一统；

---

## c10

**c10是什么？**      
来自坊间，有人说是**C**affe **TEN**sor library，也有人说是**C**ore **TEN**sor library，根据[github/pytorch issue 14850](https://github.com/pytorch/pytorch/issues/14850)的描述，core tensor library更为接近，但是c10里的这个10，指代的是cuda10的意思，因为写这个库的时候，正式cuda10大规模使用的时候，即使以后cuda11、cuda12来了，c10还是叫c10；

该源码文件夹下存放的都是最最基础的tensor库的代码，这个模块应该是对tensor相关操作的高度抽象，aten和caffe2源码中关于tensor的类似操作都迁移至此（比如registry函数），看起来这个c10应该是pytorch合并了caffe2后，对各自tensor操作函数的一次集中，很有可读性；

- c10/core
- c10/cuda
- c10/mobile

随便看一下core部分中关于device type的声明：
```
enum class DeviceType : int16_t {
  CPU = 0,
  CUDA = 1, // CUDA.
  MKLDNN = 2, // Reserved for explicit MKLDNN
  OPENGL = 3, // OpenGL
  OPENCL = 4, // OpenCL
  IDEEP = 5, // IDEEP.
  HIP = 6, // AMD HIP
  FPGA = 7, // FPGA
  MSNPU = 8, // MSNPU
  XLA = 9, // XLA / TPU
  Vulkan = 10, // Vulkan
  // NB: If you add more devices:
  //  - Change the implementations of DeviceTypeName and isValidDeviceType
  //    in DeviceType.cpp
  //  - Change the number below
  COMPILE_TIME_MAX_DEVICE_TYPES = 11,
  ONLY_FOR_TEST = 20901, // This device type is only for test.
};

constexpr DeviceType kCPU = DeviceType::CPU;
constexpr DeviceType kCUDA = DeviceType::CUDA;
constexpr DeviceType kHIP = DeviceType::HIP;
constexpr DeviceType kFPGA = DeviceType::FPGA;
constexpr DeviceType kMSNPU = DeviceType::MSNPU;
constexpr DeviceType kXLA = DeviceType::XLA;
constexpr DeviceType kVulkan = DeviceType::Vulkan;```
```
由此可见，pytorch也是希望能支持市面上的主流深度学习处理器的，英伟达(CUDA)、英特尔(MKL/Vulkan)、AMD(HIP)、谷歌(TPU/XLA)甚至是FPGA，都悉数在列。不过看起来，pytorch对ARM、RISK-V等，貌似不感冒。


类似于aten，c10的函数也基本没有机会被上层应用直接调用，他们更多的是在你增加新算子或者tensor相关函数时被用到。


## torch

先看一下torch子目录：    
![torch folder](./images/libtorch_torch_folder.png)

里面有很多的python文件，其实这里就是pytorch很多python类以及函数实现的地方。    

此处我们用的是libtorch，所以不会太过多关心python函数的实现细节。    


众所周知，pytorch的很多核心函数，比如tensor的操作，基本都是由底层c++语言实现的，因此很多人会比较关心，python接口函数最终是如何调用到底层的c++实现的。此处我们简单探索一下：
网上的文章多是认为上述功能是由pybind11来实现的，但是我们看一下源码：`{pytorch-code}/torch/library.h`，会发现，其实python函数是通过两个宏定义来完成对底层的调用的：`TORCH_LIBRARY`和`TORCH_LIBRARY_IMPL`。为啥用这个呢？是因为开发者不仅需要完成python对c++的调用，还需要完成TorchScript对c++的调用，因此，单一的pybind11就不够用了。什么是TorchScript？这是python的子集，主要的应用场景是把Python/PyTorch代码转换成等价的C++代码从而提高深度学习模型在线上生产环境部署的运行效率。Python代码会被编译成TorchScript编译器可以理解的一种格式（ScriptModule），C++的生产环境可以载入该格式的文件并用内置的JIT来执行对应的代码。[注5](https://zhuanlan.zhihu.com/p/136585481)

后面有机会我们再详细展开讨论。

我们需要重点关注的是`csrc`文件夹中的代码，如代码readme所述：

“The csrc directory contains all of the code concerned with integration with Python. This is in contrast to lib, which contains the Torch libraries that are Python agnostic. csrc depends on lib, but not vice versa.”

大部分pytorch功能即在此处完成。

有个小问题，torch::Tensor是在哪里定义的？    
程序中，我们调用的一般都是torch::Tensor，而并不是at::Tensor，文档上也说了，torch的Tensor是基于ATen的，那torch::Tensor是在哪里定义的？    

仔细看一下`{pytorch-code}/torch/csrc/api/include/torch/types.h`，可以发现，其实torch namespace包含了at、c10等namespace的声明：
![torch namespace](./images/torch_tensor.png)



<br />
<br />


## 其它组成部分：

`caffe2`：当前版本中该部分源码的作用，还在探索中。。。

来自贾扬清在知乎的回答：
### Caffe2 + PyTorch = PyTorch 1.0
![caffe2](./images/merge_caffe2.png)


<br />
<br />
<br />
<br />
<br />
* 来源：

文章：[如何有效地阅读PyTorch的源代码](https://www.zhihu.com/question/58253344)      
文章：[PyTorch源码浅析](https://zhuanlan.zhihu.com/p/34629243)    
文章：[Pytorch源码编译简明指南](https://oldpan.me/archives/pytorch-build-simple-instruction)    
文章：[PyTorch ATen代码的动态生成
](https://zhuanlan.zhihu.com/p/55966063)    
专栏：[PyTorch 源码解读](https://www.zhihu.com/column/c_1316816403623084032)    


# 3.代码量统计
<br />

```
---------------------------------------------------------------------------------------------
| Language                       |     files     |     blank    |    comment     |      code|  
---------------------------------------------------------------------------------------------
|C++                             |      2110     |     72163    |      41328     |    693175|
|Python                          |      1499     |     72683    |      71247     |    321019|
|C/C++ Header                    |      1995     |     45971    |      51072     |    248610|
|CUDA                            |       477     |     11298    |       6618     |     79523|
|C                               |       178     |      2941    |       1749     |     27841|
---------------------------------------------------------------------------------------------
|YAML                            |        39     |      2535    |       1027     |     30754|
|CMake                           |       170     |      2230    |       3571     |     14056|
|Markdown                        |       103     |      2469    |          0     |      9170|
|reStructuredText                |        85     |      3965    |       4685     |      8895|
|Bourne Shell                    |       161     |      1558    |       1612     |      6618|
|Objective-C++                   |         8     |       718    |        286     |      6377|
|Java                            |        44     |       929    |        769     |      4760|
|Assembly                        |        20     |      1299    |       1823     |      3760|
|Bazel                           |         3     |       127    |         27     |      2155|
|CSS                             |         5     |       289    |         90     |      1378|
|Gradle                          |        22     |       199    |        191     |      1071|
|SVG                             |         4     |         0    |          1     |       954|
|Dockerfile                      |        19     |       190    |        179     |       832|
|Starlark                        |         6     |        46    |         25     |       663|
|XML                             |        21     |        15    |         12     |       590|
|GLSL                            |        17     |        61    |          0     |       587|
|DOS Batch                       |        18     |       166    |          3     |       574|
|Protocol Buffers                |         7     |       132    |        356     |       402|
|INI                             |         2     |       128    |          0     |       293|
|HTML                            |         5     |        27    |         33     |       259|
|JSON                            |         4     |         0    |          0     |       231|
|Ruby                            |         3     |        10    |          3     |       184|
|make                            |         5     |        39    |         19     |       111|
|Windows Module Definition       |         1     |        20    |          0     |        60|
|PowerShell                      |         2     |         4    |          0     |        47|
|LLVM IR                         |         1     |         6    |          0     |        38|
|Lua                             |         1     |         5    |          0     |        28|
|Objective-C                     |         2     |        18    |          8     |        25|
|Bourne Again Shell              |         1     |         3    |          0     |        22|
|vim script                      |         1     |         0    |          0     |         3|
---------------------------------------------------------------------------------------------
|SUM:                            |      7039     |    222244    |     186734     |   1465065|
---------------------------------------------------------------------------------------------
```



上面是针对pytorch 1.7.1（git tag）进行的统计，其中与c/c++/cuda相关的文件超过了4500，占绝对主力。

# 4. 关于编译

目前看的不多，先简单整理一下，后面再进一步补充。
根据makefile，libtorch的编译顺序大致是：

third_party -> c10 -> caffe2 -> torch/csrc -> aten/src/TH -> aten/src/ATen -> aten/src/THC。